## Discretization

In case of the column, there are several options for adapting the spatial discretization of the PDE model.
However, the two most important ones are the number of grid cells in the column (axial direction) and the particles.
Since the lumped rate model without pores does not have particles, we only need to specify axial cells `n_col`.
The default is $100$ which should work for most scenarios.


```{note}
CADET-Core by default uses a finite volume scheme for the spatial discretization.
However, we have with Version 5.0 implemented a new method, the *Discontinuous Galerkin* method which can increase speed substantially.
```

In [ ]:
from CADETProcess.simulator import Cadet
simulator = Cadet()

simulation_results = simulator.simulate(process)

### High discretization

### Low discretization

## Visualization

Additionally to the solution at the inlet and outlet of a unit operation, we can also take a look inside the column to see the peak move.

For this purpose, set the flag in the unit's `SolutionRecorder`.
Then, the `SimulationResults` will also contain an entry for the bulk.

**Note:** Since this solution is two-dimensinal (space and time), the solution can be plotted at a given position (`plot_at_location`) or a given time (`plot_at_time`).